In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pickle
import itertools
import pandas as pd
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

In [ ]:
def addFCLayers1(model):
    output = GlobalAveragePooling2D()(model.output)
    output = Dense(512, activation='relu')(output)
    pred = Dense(12, activation='softmax')(output)

    # Create graph of new model
    return Model(input=model.input, output=pred)


def addFCLayers2(model):
    output = Flatten()(model.output)
    output = Dense(1024, activation="relu")(output)
    output = Dropout(0.2)(output)
    output = Dense(512, activation="relu")(output)
    pred = Dense(12, activation="softmax")(output)

    # Create graph of new model
    return Model(input=model.input, output=pred)


def addFCLayers3(model):
    output = Flatten()(model.output)
    output = Dense(512, activation='relu')(output)
    output = Dropout(0.2)(output)
    output = BatchNormalization()(output)
    pred = Dense(12, activation='softmax')(output)

    # Create graph of new model
    return Model(input=model.input, output=pred)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """ 
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    if normalize == False:
        plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.show()


def get_confusion_matrix(model):
    """ 'model' should be the final trained model """

    # load test data
    test_gen = ImageDataGenerator().flow_from_directory(
        './data/testing/',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

    # predict test data
    pred_labels = model4.predict_generator(test_gen)
    y_pred = np.argmax(pred_labels, axis=1)  # get highest prob class
    true_labels = test_gen.classes  # get true labels

    # list of class names
    classNames = sorted(set([cls.split('/')[0] for cls in test_gen.filenames]))

    # convert index of classes to class names
    le = LabelEncoder()
    le.fit(classNames)
    y_labels = le.inverse_transform(test_gen.classes)
    y_pred = le.inverse_transform(y_pred)

    return confusion_matrix(y_labels, y_pred), classNames


def plotMetrics(data, title):
    # summarize history for accuracy
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(data['acc'])
    plt.plot(data['val_acc'])
    plt.title('Model Accuracy ({})'.format(title))
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='lower right')

    # summarize history for loss
    plt.subplot(1, 2, 2)
    plt.plot(data['loss'])
    plt.plot(data['val_loss'])
    plt.title('Model Loss ({})'.format(title))
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper right')

    print("Final training acc: {}, loss: {}".format(data['acc'][-1], data['loss'][-1]))
    print("Final validation acc: {}, loss: {}".format(data['val_acc'][-1], data['val_loss'][-1]))

    plt.show()


def bestOptimizer(model,
                  title,
                  epochs,
                  batch_size,
                  n_train_samples,
                  n_validation_samples,
                  callback,
                  train_gen,
                  validation_gen,
                  test_gen):
    ########## ADAM ##########
    model = addFCLayers3(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_adam.h5')
    with open(title + '_adam_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

    ########## SGD ##########
    model = addFCLayers3(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True),
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_SGD.h5')
    with open(title + '_SGD_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

    ########## RMSPROP ##########
    model = addFCLayers3(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_rmsprop.h5')
    with open(title + '_rmsprop_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)


def bestLR(model,
           title,
           lrRange,
           optim,
           epochs,
           batch_size,
           n_train_samples,
           n_validation_samples,
           callback,
           train_gen,
           validation_gen,
           test_gen):
    for lr in lrRange:
        print("---> Learning rate: {}".format(lr))
        model = addFCLayers3(model)
        # choose correct optimizer
        if optim == 'Adam':
            opt = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        elif optim == 'SGD':
            opt = optimizers.SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=True)
        elif optim == 'RMSprop':
            opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
        hist = model.fit_generator(
            train_gen,
            steps_per_epoch=n_train_samples // batch_size,
            epochs=epochs,
            validation_data=validation_gen,
            callbacks=[callback],
            validation_steps=n_validation_samples // batch_size)
        test_data = model.evaluate_generator(test_gen)
        hist.history['test'] = test_data
        model.save(title + '_lr_{}.h5'.format(lr))
        with open(title + '_lr_{}_hist.pkl'.format(lr), 'wb') as file_pi:
            pickle.dump(hist.history, file_pi)


def bestFCLayer(model,
                title,
                lr,
                optim,
                epochs,
                batch_size,
                n_train_samples,
                n_validation_samples,
                callback,
                train_gen,
                validation_gen,
                test_gen):
    # choose correct optimizer
    if optim == 'Adam':
        opt = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    elif optim == 'SGD':
        opt = optimizers.SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=True)
    elif optim == 'RMSprop':
        opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)

    ########## FC1 ##########
    model = addFCLayers1(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_fc1.h5')
    with open(title + '_fc1_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

    ########## FC2 ##########
    model = addFCLayers2(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_fc2.h5')
    with open(title + '_fc2_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

    ########## FC3 ##########
    model = addFCLayers3(model)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    hist = model.fit_generator(
        train_gen,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_gen,
        callbacks=[callback],
        validation_steps=n_validation_samples // batch_size)
    test_data = model.evaluate_generator(test_gen)
    hist.history['test'] = test_data
    model.save(title + '_fc3.h5')
    with open(title + '_fc3_hist.pkl', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

In [ ]:
# Parameters
batch_size = 32
epochs = 1000

##################################################################
#                       SETUP DATA GENERATORS
##################################################################
print("Setting up data generators...")
# init real-time data augmentation parameters
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
)

train_gen = datagen.flow_from_directory(
    './data/training',
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')
validation_gen = ImageDataGenerator().flow_from_directory(
    './data/validation',
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')
test_gen = ImageDataGenerator().flow_from_directory(
    './data/testing/',
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

# Init early stopping functionality
callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

##################################################################
#                       SETUP MODELS
##################################################################
print("Setting up models...")
# VGG16
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(299, 299, 3), pooling=None)
for layer in vgg16.layers:  # do not train
    layer.trainable = False

# VGG19
vgg19 = VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=(299, 299, 3), pooling=None)
for layer in vgg19.layers:  # do not train
    layer.trainable = False

# # Xception
xception = Xception(include_top=False, input_tensor=None, input_shape=(299, 299, 3), pooling='None')
for layer in xception.layers:  # do not train
    layer.trainable = False

##################################################################
#                       BEST OPTIMIZER
##################################################################
print("------ Best Optimizer ------")

n_train_samples = 12210
n_validation_samples = 2572

### VGG16
ut.bestOptimizer(vgg16, 'vgg16', epochs, batch_size, n_train_samples, n_validation_samples, callback, train_gen,
                 validation_gen, test_gen)

### VGG19
ut.bestOptimizer(vgg19, 'vgg19', epochs, batch_size, n_train_samples, n_validation_samples, callback, train_gen,
                 validation_gen, test_gen)

### VGG19
ut.bestOptimizer(xception, 'xception', epochs, batch_size, n_train_samples, n_validation_samples, callback, train_gen,
                 validation_gen, test_gen)

##################################################################
#                       BEST LEARNING RATE
##################################################################
print("------ Best learning rate ------")

lrRange = [0.1, 0.01, 0.001, 0.0001]

### VGG16
ut.bestLR(vgg16, 'vgg16', lrRange, 'SGD', epochs, batch_size, n_train_samples, n_validation_samples, callback,
          train_gen, validation_gen, test_gen)

### VGG19
ut.bestLR(vgg19, 'vgg19', lrRange, 'Adam', epochs, batch_size, n_train_samples, n_validation_samples, callback,
          train_gen, validation_gen, test_gen)

### VGG19
ut.bestLR(xception, 'xception', lrRange, 'SGD', epochs, batch_size, n_train_samples, n_validation_samples, callback,
          train_gen, validation_gen, test_gen)

##################################################################
#                       BEST CLASSIFICATION LAYER
##################################################################
print("------ Best classification layer ------")

### VGG16
ut.bestFCLayer(vgg16, 'vgg16', .01, 'SGD', epochs, batch_size, n_train_samples, n_validation_samples, callback,
               train_gen, validation_gen, test_gen)

### VGG19
ut.bestFCLayer(vgg19, 'vgg19', .001, 'Adam', epochs, batch_size, n_train_samples, n_validation_samples, callback,
               train_gen, validation_gen, test_gen)

### VGG19
ut.bestFCLayer(xception, 'xception', .01, 'SGD', epochs, batch_size, n_train_samples, n_validation_samples, callback,
               train_gen, validation_gen, test_gen)

##################################################################
#                       PLOT CONFUSION MATRIX
##################################################################
print("------ Plot confusion matrix ------")

### Load your model first! 
model = addFCLayers3(xception)
model.load_weights('xception_fc3.h5')
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=.01, momentum=0.0, decay=0.0, nesterov=True),
              metrics=['accuracy'])

cm, classNames = ut.get_confusion_matrix(model)
ut.plot_confusion_matrix(cm, classNames, normalize=True, title='Confusion Matrix for Xception')